# Medicare Provider Search Tool
Notebook to search for the lowest cost medicare providers for a given condition.

In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.1f}'.format
from math import radians, cos, sin, asin, sqrt
from IPython.display import display

#Load data into dataframes. Files must be in same folder as main program.
df = pd.read_csv('InpatientCharges.csv')
drg2mdc = pd.read_csv('drg2mdcxw2014.csv')
df_ziplatlon = pd.read_csv('zip_latlon.csv')
df_ziplatlon.set_index('zip', inplace=True)

#Need to strip currency formatting for charges
for item in ['Average Covered Charges', 'Average Total Payments', 'Average Medicare Payments']:
    df[item] = (df[item].replace( '[\$,)]','', regex=True )
                        .replace( '[(]','-',   regex=True ).astype(float))

#Separate DRG Codes (first 3 digits)
df['DRG_Code'] = df['DRG Definition'].apply(lambda x: int(x[0:3])) #Int because lookup table uses ints

#Join the two tables, adding columns mdc and mdcdesc to df
df = df.join(drg2mdc.set_index('drg'), on='DRG_Code', how='left')

#Elim White Spaces in column names
df.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)

def haversine(lon1, lat1, lon2, lat2):
    '''
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    '''
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3956 # Radius of earth in kilometers. Use 3956 for miles, 6371 for km
    return c * r

def zip_dist(zip1, zip2):
    '''Run the haversine forumla with zip code inputs instead of lat & lon
    '''
    return haversine(df_ziplatlon.loc[zip1][1],
                     df_ziplatlon.loc[zip1][0],
                     df_ziplatlon.loc[zip2][1],
                     df_ziplatlon.loc[zip2][0])

def wavg(group, avg_name, weight_name):
    ''' http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
    In rare instance, we may not have weights, so just return the mean. Customize this if your business case
    should return otherwise.
    '''
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [3]:
print('Welcome to the Medicare Provider Search Tool!\n')
print('This program takes your zip code, a medical treatment category, and the distance you are willing to travel,')
print('and returns a list of hospitals that have provided treatment in the selected category to Medicare patients.')
print('The list is sorted by lowest to highest average payments the hospital receives for the specified treatment category.')

print('\nNOTE: Individual costs vary widely based on numerous factors. The costs in this table only provide an average.')
print('Your costs may be significantly higher or lower than the figures reported here.')

print('\nNOTE: Distances are calculated from the geographic center of a zip code; therefore, the distance')
print('from your location to the provider is not exact.')

#Omitted: Error checking, looping, running from python command line

#Input zip
user_zip = int(input('\nPlease enter your zip code.\n'))
print()

#Print medical conditions, solicit input
print('Here are a list of medical conditions.')
print(df.mdcdesc.unique())
user_mdc = int(input('\nPlease enter the number of the medical category for which you require treatment.\n'))

#Input radius
user_rad = float(input('\nPlease enter the distance, in miles, you would be willing to travel to receive treatment.\n'))

#Perform calculations
#Generate df of all zip codes and their distance to user zip code
zip_index = []
dist = []
for index, row in df_ziplatlon.iterrows():  #Parallel processing would yield big speed benefit.
    zip_index.append(index)
    dist.append(zip_dist(user_zip, index))

df_zipdist = pd.DataFrame({'zip':zip_index, 'Distance':dist})

#Reduce zips to those within radius
df_zipdist = df_zipdist[df_zipdist.Distance <= user_rad]

#Join zips within radius with main df; inner join b/c we want only where zips exist in both df's
df_user = df.join(df_zipdist.set_index('zip'), on='Provider_Zip_Code', how='inner')

#Filter for only those where MDC matches user input
df_user = df_user[df_user.mdc == user_mdc]

if len(df_user) == 0:
    print('Your search parameters returned 0 results. Either you entered an invalid medical condition,')
    print('or there were no providers for the medical condition you selected within the distance provided.')
    print('Try entering a larger distance.')
else:
    #Calculate weighted averages for total payments
    df_wt = df_user.groupby('Provider_Id').apply(wavg, 'Average_Total_Payments', 'Total_Discharges').to_frame()
    df_wt.columns = ['Weighted Avg. Cost']

    #Join back into main data
    final_cols= ['Distance', 'Provider_Id', 'Provider_Name', 'Provider_Street_Address', 'Provider_City',
                 'Provider_State', 'Provider_Zip_Code']

    print('\nHere is your list of providers, sorted by lowest to highest cost.')
    print('NOTE: Individual costs vary widely based on numerous factors. The costs in this table only provide an average.')
    print('Your costs may be significantly higher or lower than the figures reported here.')
    df_user = df_user[final_cols]
    df_user = df_wt.join(df_user.set_index('Provider_Id'), how='left') \
                   .groupby('Provider_Id').first().sort_values('Weighted Avg. Cost')
                    #.first() gets just the first row of the group (all rows within the group are identical)
                    #.drop_duplicates(subset='A') might have worked too.
    
    #Format cost column
    df_user['Weighted Avg. Cost'] = df_user['Weighted Avg. Cost'].map('${:,.0f}'.format)
    
    #Display output
    print('\nYour search returned {} results'.format(len(df_user)))
    display(df_user.set_index('Provider_Name'))

print('\nThank you for using the Medicare Provider Search Tool!')

Welcome to the Medicare Provider Search Tool!

This program takes your zip code, a medical treatment category, and the distance you are willing to travel,
and returns a list of hospitals that have provided treatment in the selected category to Medicare patients.
The list is sorted by lowest to highest average payments the hospital receives for the specified treatment category.

NOTE: Individual costs vary widely based on numerous factors. The costs in this table only provide an average.
Your costs may be significantly higher or lower than the figures reported here.

NOTE: Distances are calculated from the geographic center of a zip code; therefore, the distance
from your location to the provider is not exact.

Please enter your zip code.
80401

Here are a list of medical conditions.
['1: DISEASES & DISORDERS OF THE NERVOUS SYSTEM'
 '3: DISEASES & DISORDERS OF THE EAR, NOSE, MOUTH & THROAT'
 '4: DISEASES & DISORDERS OF THE RESPIRATORY SYSTEM'
 '5: DISEASES & DISORDERS OF THE CIRCULATORY

,Weighted Avg. Cost,Distance,Provider_Street_Address,Provider_City,Provider_State,Provider_Zip_Code
Provider_Name,,,,,,
SKY RIDGE MEDICAL CENTER,"$5,478",22.4,10101 RIDGE GATE PARKWAY,LONE TREE,CO,80124
EXEMPLA LUTHERAN MEDICAL CENTER,"$5,939",7.9,8300 W 38TH AVE,WHEAT RIDGE,CO,80033
CENTURA HEALTH-ST ANTHONY NORTH HOSPITAL,"$6,152",15.0,2551 W 84TH AVENUE,WESTMINSTER,CO,80031
CENTURA HEALTH-LITTLETON ADVENTIST HOSPITAL,"$6,220",17.7,7700 S BROADWAY,LITTLETON,CO,80122
BOULDER COMMUNITY HOSPITAL,"$6,220",22.7,1100 BALSAM AVENUE,BOULDER,CO,80304
EXEMPLA GOOD SAMARITAN MEDICAL CENTER LLC,"$6,374",21.5,200 EXEMPLA CIRCLE,LAFAYETTE,CO,80026
CENTURA HEALTH-PORTER ADVENTIST HOSPITAL,"$6,894",14.8,2525 S DOWNING ST,DENVER,CO,80210
SWEDISH MEDICAL CENTER,"$7,336",15.3,501 E HAMPDEN AVENUE,ENGLEWOOD,CO,80113
"MEDICAL CENTER OF AURORA, THE","$7,417",21.2,1501 S POTOMAC ST,AURORA,CO,80012



Thank you for using the Medicare Provider Search Tool!
